# Scraping the Election Results from the CQ Press Library Website

All the congressional election results data we need is stored on the CQ Press Library Website. The website is behind a username/password wall (our Georgetown credentials), so for this notebook we will use one of the html pages that was previously saved (Alabama's congressional election information). 

This notebook will walk through the process of scraping the html page, storing the relevant information as a dataframe, initial required cleaning, wrangling, and munging, and saving as a .csv so that it is ready to be ingested into the database.

## Getting the data.

The data we are looking for is stored by congressional election year, by state, on the CQ Press Library Website. We need data for the 2012 and 2014 election years, in each of the fifty states. Each webpage includes the election information for a single state, for both the 2012 and 2014 cycles. 

In [1]:
# import the necessary libraries
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# create handle for BeautifulSoup instance
soup = BeautifulSoup(open("./Data/CQ Voting and Elections Collection.html"), "html.parser")

 Next we need to pull all the table headers and table data from the page. 

In [3]:
table = soup.findAll('td')

In [4]:
#take a look at what we've extracted

table

[<td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Republican</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Democratic</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Highest Other</span></td>,
 <td align="center" valign="top"><span class="tablehead">Other</span></td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="4" valign="top"><span class="tablehead">Total Vote (%)</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Major Party Vote (%)</span></td>,
 <td align="center" valign="top"><span class="tablehead">Year</span></td>,
 <td align="center" valign="top"><span class="tablehead">CD</span></td>,
 <td align="center" valign="top"><span class="tablehead">Redistricted Date</span></td>,
 <td align="center" valign="top"><span class="tabl

The table headers look like they comprise the first two rows, so we need to take a look at them to make sure that we're capturing all the information we want to me. 

In [5]:
#pull out just the first row to see the headers

soup.findAll('tr')[1].findAll('td')

[<td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Republican</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Democratic</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Highest Other</span></td>,
 <td align="center" valign="top"><span class="tablehead">Other</span></td>,
 <td valign="top"> </td>,
 <td valign="top"> </td>,
 <td align="center" colspan="4" valign="top"><span class="tablehead">Total Vote (%)</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Major Party Vote (%)</span></td>]

In [6]:
#but the second row of headers are what we actually care about. 

soup.findAll('tr')[2].findAll('td')

[<td align="center" valign="top"><span class="tablehead">Year</span></td>,
 <td align="center" valign="top"><span class="tablehead">CD</span></td>,
 <td align="center" valign="top"><span class="tablehead">Redistricted Date</span></td>,
 <td align="center" valign="top"><span class="tablehead">Total Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Candidate</span></td>,
 <td align="center" valign="top"><span class="tablehead">Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Candidate</span></td>,
 <td align="center" valign="top"><span class="tablehead">Vote</span></td>,
 <td align="center" valign="top"><span class="tablehead">Candidate</span></td>,
 <td align="center" valign="top"><span class="tablehead">Other Vote</span></td>,
 <td align="center" colspan="2" valign="top"><span class="tablehead">Plurality</span></td>,
 <td align="center" valign="top"><span class=

In [7]:
#extract just the text, and store each label in a list called column_headers
column_headers = [td.getText() for td in 
                  soup.findAll('tr')[2].findAll('td')]

In [8]:
#take a look
column_headers

['Year',
 'CD',
 'Redistricted Date',
 'Total Vote',
 'Vote',
 'Candidate',
 'Vote',
 'Candidate',
 'Vote',
 'Candidate',
 'Other Vote',
 'Plurality',
 'Rep.',
 'Dem.',
 'Highest Other',
 'Other',
 'Rep.',
 'Dem.']

Modify the column headers so that the column headers include the first row from the html table

In [9]:
column_headers[4] = "Republican Vote"
column_headers[5] = "Republican Candidate"
column_headers[6] = "Democratic Vote"
column_headers[7] = "Democratic Candidate"
column_headers[8] = "Highest Other Vote"
column_headers[9] = "Highest Other Candidate"
column_headers[12] = "Total Vote Percent Rep"
column_headers[13] = "Total Vote Percent Dem"
column_headers[14] = "Total Vote Percent Highest Other"
column_headers[15] = "Total Vote Percent Other"
column_headers[16] = "Major Party Vote Percent Rep"
column_headers[17] = "Major Party Vote Percent Dem"

column_headers

['Year',
 'CD',
 'Redistricted Date',
 'Total Vote',
 'Republican Vote',
 'Republican Candidate',
 'Democratic Vote',
 'Democratic Candidate',
 'Highest Other Vote',
 'Highest Other Candidate',
 'Other Vote',
 'Plurality',
 'Total Vote Percent Rep',
 'Total Vote Percent Dem',
 'Total Vote Percent Highest Other',
 'Total Vote Percent Other',
 'Major Party Vote Percent Rep',
 'Major Party Vote Percent Dem']

In [10]:
#subset the table to just include the data (rows three onwards), and store in data_rows
data_rows = soup.findAll('tr')[3:]

In [11]:
#take a look
data_rows

[<tr> <td align="left" valign="top"><span class="tabletext">2014</span></td> <td align="right" valign="top"><span class="tabletext">District 1</span></td> <td align="right" valign="top"><span class="tabletext">01/01/2012</span></td> <td align="right" valign="top"><span class="tabletext">152,234</span></td> <td align="right" valign="top"><span class="tabletext">103,758</span></td> <td align="right" valign="top"><span class="tabletext">Byrne, Bradley [<a href="http://library.cqpress.com.proxy.library.georgetown.edu/elections/document.php?id=avg2012-3AL3&amp;type=toc&amp;num=9676B457-9FD2-4541-985C-45CAE50EE210#incumfn">*</a>]</span></td> <td align="right" valign="top"><span class="tabletext">48,278</span></td> <td align="right" valign="top"><span class="tabletext">LeFlore, Burton R.</span></td> <td align="right" valign="top"><span class="tabletext">198</span></td> <td align="right" valign="top"><span class="tabletext">Write-In(Write-in)</span></td> <td align="right" valign="top"><span cl

In [12]:
type(data_rows)

list

In [13]:
#extract only the data from the table sub-set, and store in race_data
race_data = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [14]:
#take a look
race_data

[['2014',
  'District 1',
  '01/01/2012',
  '152,234',
  '103,758',
  'Byrne, Bradley [*]',
  '48,278',
  'LeFlore, Burton R.',
  '198',
  'Write-In(Write-in)',
  'N/A',
  '55,480',
  'R',
  '68.16',
  '31.71',
  '0.13',
  '0',
  '68.25',
  '31.75'],
 ['2012',
  'District 1',
  '01/01/2012',
  'Unopposed',
  '196,374',
  'Bonner, Josiah Robbins Jr. [*]',
  'N/A',
  'N/A',
  '4,302',
  'Scattered Write-In(Write-in)',
  'N/A',
  '192,072',
  'R',
  '97.86',
  '0',
  '2.14',
  '0',
  '100',
  '0'],
 ['2014',
  'District 2',
  '01/01/2012',
  '167,952',
  '113,103',
  'Roby, Martha [*]',
  '54,692',
  'Wright, Erick',
  '157',
  'Write-In(Write-in)',
  'N/A',
  '58,411',
  'R',
  '67.34',
  '32.56',
  '0.09',
  '0',
  '67.41',
  '32.59'],
 ['2012',
  'District 2',
  '01/01/2012',
  '283,953',
  '180,591',
  'Roby, Martha [*]',
  '103,092',
  'Ford, Therese',
  '270',
  'Scattered Write-In(Write-in)',
  'N/A',
  '77,499',
  'R',
  '63.6',
  '36.31',
  '0.1',
  '0',
  '63.66',
  '36.34'],
 [

In [15]:
#combine the headers with the data in one dataframe
df = pd.DataFrame(race_data, columns=column_headers)

AssertionError: 18 columns passed, passed data had 19 columns

We got an error because there were fewer column headers than were columns of data. Looking at the original source html, that is because there is a party column that has no title. So we need to add that column where the data is (in between Plurality and Rep)

In [16]:
column_headers.insert(12, "WinningParty")

In [17]:
column_headers

['Year',
 'CD',
 'Redistricted Date',
 'Total Vote',
 'Republican Vote',
 'Republican Candidate',
 'Democratic Vote',
 'Democratic Candidate',
 'Highest Other Vote',
 'Highest Other Candidate',
 'Other Vote',
 'Plurality',
 'WinningParty',
 'Total Vote Percent Rep',
 'Total Vote Percent Dem',
 'Total Vote Percent Highest Other',
 'Total Vote Percent Other',
 'Major Party Vote Percent Rep',
 'Major Party Vote Percent Dem']

In [18]:
df = pd.DataFrame(race_data, columns=column_headers)

In [19]:
df

,Year,CD,Redistricted Date,Total Vote,Republican Vote,Republican Candidate,Democratic Vote,Democratic Candidate,Highest Other Vote,Highest Other Candidate,Other Vote,Plurality,WinningParty,Total Vote Percent Rep,Total Vote Percent Dem,Total Vote Percent Highest Other,Total Vote Percent Other,Major Party Vote Percent Rep,Major Party Vote Percent Dem
0,2014,District 1,01/01/2012,"152,234","103,758","Byrne, Bradley [*]","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,District 1,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr. [*]",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,District 2,01/01/2012,"167,952","113,103","Roby, Martha [*]","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,District 2,01/01/2012,"283,953","180,591","Roby, Martha [*]","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,District 3,01/01/2012,"156,620","103,558","Rogers, Mike D. [*]","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,District 3,01/01/2012,"273,930","175,306","Rogers, Mike D. [*]","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,District 4,01/01/2012,Unopposed,"132,831","Aderholt, Robert [*]",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,District 4,01/01/2012,"269,118","199,071","Aderholt, Robert [*]","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],District 5,01/01/2012,"154,974","115,338","Brooks, Mo [*]",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,District 5,01/01/2012,"291,293","189,185","Brooks, Mo [*]","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


## Munging the Data

We have the raw data from the html into a data frame, now we need to clean the data to ready it for ingestion. This includes several steps:

* Delete the empty record at the end.
* Add a column for State, and fill in with State's abbreviation.
* Delete the word *District* in the District columns. 
* Delete asterisks from all Candidate Name columns.
* Drop Total Vote Percent Other column.

In [20]:
df = df[:-1]

In [21]:
df

,Year,CD,Redistricted Date,Total Vote,Republican Vote,Republican Candidate,Democratic Vote,Democratic Candidate,Highest Other Vote,Highest Other Candidate,Other Vote,Plurality,WinningParty,Total Vote Percent Rep,Total Vote Percent Dem,Total Vote Percent Highest Other,Total Vote Percent Other,Major Party Vote Percent Rep,Major Party Vote Percent Dem
0,2014,District 1,01/01/2012,"152,234","103,758","Byrne, Bradley [*]","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,District 1,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr. [*]",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,District 2,01/01/2012,"167,952","113,103","Roby, Martha [*]","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,District 2,01/01/2012,"283,953","180,591","Roby, Martha [*]","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,District 3,01/01/2012,"156,620","103,558","Rogers, Mike D. [*]","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,District 3,01/01/2012,"273,930","175,306","Rogers, Mike D. [*]","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,District 4,01/01/2012,Unopposed,"132,831","Aderholt, Robert [*]",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,District 4,01/01/2012,"269,118","199,071","Aderholt, Robert [*]","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],District 5,01/01/2012,"154,974","115,338","Brooks, Mo [*]",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,District 5,01/01/2012,"291,293","189,185","Brooks, Mo [*]","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


In [22]:
#add a state column, fill in with AL
df.insert(2 , "State", "AL", allow_duplicates=False)


In [23]:
df

,Year,CD,State,Redistricted Date,Total Vote,Republican Vote,Republican Candidate,Democratic Vote,Democratic Candidate,Highest Other Vote,Highest Other Candidate,Other Vote,Plurality,WinningParty,Total Vote Percent Rep,Total Vote Percent Dem,Total Vote Percent Highest Other,Total Vote Percent Other,Major Party Vote Percent Rep,Major Party Vote Percent Dem
0,2014,District 1,AL,01/01/2012,"152,234","103,758","Byrne, Bradley [*]","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,District 1,AL,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr. [*]",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,District 2,AL,01/01/2012,"167,952","113,103","Roby, Martha [*]","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,District 2,AL,01/01/2012,"283,953","180,591","Roby, Martha [*]","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,District 3,AL,01/01/2012,"156,620","103,558","Rogers, Mike D. [*]","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,District 3,AL,01/01/2012,"273,930","175,306","Rogers, Mike D. [*]","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,District 4,AL,01/01/2012,Unopposed,"132,831","Aderholt, Robert [*]",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,District 4,AL,01/01/2012,"269,118","199,071","Aderholt, Robert [*]","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],District 5,AL,01/01/2012,"154,974","115,338","Brooks, Mo [*]",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,District 5,AL,01/01/2012,"291,293","189,185","Brooks, Mo [*]","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


In [24]:
#remove district
df['CD'] = df['CD'].str.lstrip('District')
df

C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Year,CD,State,Redistricted Date,Total Vote,Republican Vote,Republican Candidate,Democratic Vote,Democratic Candidate,Highest Other Vote,Highest Other Candidate,Other Vote,Plurality,WinningParty,Total Vote Percent Rep,Total Vote Percent Dem,Total Vote Percent Highest Other,Total Vote Percent Other,Major Party Vote Percent Rep,Major Party Vote Percent Dem
0,2014,1,AL,01/01/2012,"152,234","103,758","Byrne, Bradley [*]","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,1,AL,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr. [*]",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,2,AL,01/01/2012,"167,952","113,103","Roby, Martha [*]","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,2,AL,01/01/2012,"283,953","180,591","Roby, Martha [*]","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,3,AL,01/01/2012,"156,620","103,558","Rogers, Mike D. [*]","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,3,AL,01/01/2012,"273,930","175,306","Rogers, Mike D. [*]","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,4,AL,01/01/2012,Unopposed,"132,831","Aderholt, Robert [*]",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,4,AL,01/01/2012,"269,118","199,071","Aderholt, Robert [*]","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],5,AL,01/01/2012,"154,974","115,338","Brooks, Mo [*]",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,5,AL,01/01/2012,"291,293","189,185","Brooks, Mo [*]","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


In [25]:
#remove "[*] next to candidate names"
df['Republican Candidate'] = df['Republican Candidate'].str.rstrip('[*]')
df['Democratic Candidate'] = df['Democratic Candidate'].str.rstrip('[*]')
df['Highest Other Candidate'] = df['Highest Other Candidate'].str.rstrip('[*]')

df

C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,Year,CD,State,Redistricted Date,Total Vote,Republican Vote,Republican Candidate,Democratic Vote,Democratic Candidate,Highest Other Vote,Highest Other Candidate,Other Vote,Plurality,WinningParty,Total Vote Percent Rep,Total Vote Percent Dem,Total Vote Percent Highest Other,Total Vote Percent Other,Major Party Vote Percent Rep,Major Party Vote Percent Dem
0,2014,1,AL,01/01/2012,"152,234","103,758","Byrne, Bradley","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,0,68.25,31.75
1,2012,1,AL,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr.",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,0,100,0
2,2014,2,AL,01/01/2012,"167,952","113,103","Roby, Martha","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,0,67.41,32.59
3,2012,2,AL,01/01/2012,"283,953","180,591","Roby, Martha","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,0,63.66,36.34
4,2014,3,AL,01/01/2012,"156,620","103,558","Rogers, Mike D.","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,0,66.22,33.78
5,2012,3,AL,01/01/2012,"273,930","175,306","Rogers, Mike D.","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,0,64.11,35.89
6,2014,4,AL,01/01/2012,Unopposed,"132,831","Aderholt, Robert",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,0,100,0
7,2012,4,AL,01/01/2012,"269,118","199,071","Aderholt, Robert","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,0,74.07,25.93
8,2014 [1],5,AL,01/01/2012,"154,974","115,338","Brooks, Mo",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,0.41,100,0
9,2012,5,AL,01/01/2012,"291,293","189,185","Brooks, Mo","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,0,65.02,34.98


In [26]:
#drop the Total Vote Percent Other column
df.drop("Total Vote Percent Other", 1)

,Year,CD,State,Redistricted Date,Total Vote,Republican Vote,Republican Candidate,Democratic Vote,Democratic Candidate,Highest Other Vote,Highest Other Candidate,Other Vote,Plurality,WinningParty,Total Vote Percent Rep,Total Vote Percent Dem,Total Vote Percent Highest Other,Major Party Vote Percent Rep,Major Party Vote Percent Dem
0,2014,1,AL,01/01/2012,"152,234","103,758","Byrne, Bradley","48,278","LeFlore, Burton R.",198,Write-In(Write-in),N/A,"55,480",R,68.16,31.71,0.13,68.25,31.75
1,2012,1,AL,01/01/2012,Unopposed,"196,374","Bonner, Josiah Robbins Jr.",N/A,N/A,"4,302",Scattered Write-In(Write-in),N/A,"192,072",R,97.86,0,2.14,100,0
2,2014,2,AL,01/01/2012,"167,952","113,103","Roby, Martha","54,692","Wright, Erick",157,Write-In(Write-in),N/A,"58,411",R,67.34,32.56,0.09,67.41,32.59
3,2012,2,AL,01/01/2012,"283,953","180,591","Roby, Martha","103,092","Ford, Therese",270,Scattered Write-In(Write-in),N/A,"77,499",R,63.6,36.31,0.1,63.66,36.34
4,2014,3,AL,01/01/2012,"156,620","103,558","Rogers, Mike D.","52,816","Smith, Jesse Tremain",246,Write-In(Write-in),N/A,"50,742",R,66.12,33.72,0.16,66.22,33.78
5,2012,3,AL,01/01/2012,"273,930","175,306","Rogers, Mike D.","98,141","Harris, John Andrew",483,Scattered Write-In(Write-in),N/A,"77,165",R,64,35.83,0.18,64.11,35.89
6,2014,4,AL,01/01/2012,Unopposed,"132,831","Aderholt, Robert",N/A,N/A,"1,921",Write-In(Write-in),N/A,"130,910",R,98.57,0,1.43,100,0
7,2012,4,AL,01/01/2012,"269,118","199,071","Aderholt, Robert","69,706","Boman, Daniel H.",341,Scattered Write-In(Write-in),N/A,"129,365",R,73.97,25.9,0.13,74.07,25.93
8,2014 [1],5,AL,01/01/2012,"154,974","115,338","Brooks, Mo",N/A,N/A,"39,005","Bray, Mark(Unaffiliated)",631,"76,333",R,74.42,0,25.17,100,0
9,2012,5,AL,01/01/2012,"291,293","189,185","Brooks, Mo","101,772","Holley, Charlie L.",336,Scattered Write-In(Write-in),N/A,"87,413",R,64.95,34.94,0.12,65.02,34.98


## Ready for Ingestion into Database

In [27]:
#export into a csv

df.to_csv("Alabama.csv", sep=',')

## Next Steps

Loop through html pages of other 49 states, and concatenate results into one csv. 

Process Refinement: how to pull straight from the website, inputting username and password automatically to allow the script to fetch the data from the website itself. 